# Simulate the ground state of a Hydrogen molecule using Variational Quantum Eigensolver (VQE) on the Quantinuum emulator

In this notebook, you will run VQE for a H2 molecule on the Quantinuum emulator via Azure Quantum using Qiskit.

VQE is a variational algorithm that uses an [SPSA](https://en.wikipedia.org/wiki/Simultaneous_perturbation_stochastic_approximation) optimization loop to minimize a cost function. The cost function is an energy evaluation $H|\psi> = E|\psi>$ where $|\psi>$ is a parametric trial state that estimates the ground state of the molecule. For each evaluation, we modify the trial state until the energy reaches a minimum.

For more information about running VQE using Qiskit, see: https://qiskit.org/textbook/ch-applications/vqe-molecules.html#implementationnoisy

First, install and import the required packages.

In [ ]:
!pip install qiskit-aer

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from qiskit import IBMQ, BasicAer, Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

Prepare the qubit operators using PySCF.

In [2]:
driver = PySCFDriver(atom='H .0 .0 -0.3625; H .0 .0 0.3625', unit=UnitsType.ANGSTROM, charge=0, spin=0, basis='sto3g')
molecule = driver.run()
num_particles = molecule.num_alpha + molecule.num_beta
qubit_op = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals).mapping(map_type='parity')
qubit_op = Z2Symmetries.two_qubit_reduction(qubit_op, num_particles)

## 1. Simulate locally

You can simulate the program locally using the Aer simulator. Create a QuantumInstance with a noise model, generated using a mock device "FakeVigo" with noise characteristics.

In [3]:
from qiskit.test.mock import FakeVigo
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel

backend = AerSimulator()
device_backend = FakeVigo()
device = QasmSimulator.from_backend(device_backend)

Then, run the simulation using the VQE class.

### Simulate locally with noise and error mitigation

In [4]:
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device)
basis_gates = noise_model.basis_gates

quantum_instance = QuantumInstance(backend=backend, 
                                   shots=8192, 
                                   noise_model=noise_model, 
                                   coupling_map=coupling_map,
                                   measurement_error_mitigation_cls=CompleteMeasFitter,
                                   cals_matrix_refresh_period=30)

exact_solution = NumPyEigensolver(qubit_op).run()
print("Exact Result:", np.real(exact_solution.eigenvalues) + molecule.nuclear_repulsion_energy)
# Create optimizer
optimizer = SPSA(maxiter=100)
# Create the variational form of the ansatz. This is using the RYRZ variational form.
# For more information, see the Qiskit textbook: https://qiskit.org/textbook/ch-applications/vqe-molecules.html
var_form = EfficientSU2(qubit_op.num_qubits, entanglement="linear")
# Create a VQE object that runs the algorithm
vqe = VQE(qubit_op, var_form, optimizer=optimizer)
# Run the VQE program
ret = vqe.run(quantum_instance)
vqe_result = np.real(ret['eigenvalue']+ molecule.nuclear_repulsion_energy)
print("VQE Result:", vqe_result)

Exact Result: [-1.13722138]
VQE Result: -1.1036463255433093


The parameters found by the optimization loop:

In [5]:
p0 = ret.optimal_point
p0

array([ 2.64222138,  1.98857084, -1.70870039, -3.29137529, -1.35222182,
       -6.29435227, -1.42635834,  1.44307708,  0.37995398, -2.6046815 ,
        2.8102566 ,  0.1773668 ,  0.78413311, -1.36714306,  0.1063378 ,
       -2.03098041])

The energy was evaluated a total of `ret.cost_function_evals` times until the minimum was found.

In [6]:
ret.cost_function_evals

241

### Circuit visualization

Each energy evaluation consists of two circuits that are run. You can visualize these circuits with Qiskit using the `vqe` instance.

In [7]:
# The below function removes any wires that are automatically generated but are not used, to make it easier to read.
# https://quantumcomputing.stackexchange.com/questions/25672/remove-inactive-qubits-from-qiskit-circuit
from qiskit.converters import circuit_to_dag, dag_to_circuit
from collections import OrderedDict

def remove_idle_qwires(circ):
    dag = circuit_to_dag(circ)

    idle_wires = list(dag.idle_wires())
    for w in idle_wires:
        dag._remove_idle_wire(w)
        dag.qubits.remove(w)

    dag.qregs = OrderedDict()

    return dag_to_circuit(dag)

circs = vqe._circuit_sampler._transpiled_circ_cache
circs = [remove_idle_qwires(circ) for circ in circs]
circ = circs[0]

In [8]:
circ.draw()

┌──────────┐┌──────────┐     ┌──────────┐┌──────────┐     ┌──────────┐»
   0: ┤ Ry(θ[0]) ├┤ Rz(θ[2]) ├──■──┤ Ry(θ[4]) ├┤ Rz(θ[6]) ├──■──┤ Ry(θ[8]) ├»
      ├──────────┤├──────────┤┌─┴─┐├──────────┤├──────────┤┌─┴─┐├──────────┤»
   1: ┤ Ry(θ[1]) ├┤ Rz(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├┤ Rz(θ[7]) ├┤ X ├┤ Ry(θ[9]) ├»
      └──────────┘└──────────┘└───┘└──────────┘└──────────┘└───┘└──────────┘»
c0: 2/══════════════════════════════════════════════════════════════════════»
                                                                            »
«      ┌───────────┐     ┌───────────┐┌───────────┐┌───┐┌─┐   
«   0: ┤ Rz(θ[10]) ├──■──┤ Ry(θ[12]) ├┤ Rz(θ[14]) ├┤ H ├┤M├───
«      ├───────────┤┌─┴─┐├───────────┤├───────────┤├───┤└╥┘┌─┐
«   1: ┤ Rz(θ[11]) ├┤ X ├┤ Ry(θ[13]) ├┤ Rz(θ[15]) ├┤ H ├─╫─┤M├
«      └───────────┘└───┘└───────────┘└───────────┘└───┘ ║ └╥┘
«c0: 2/══════════════════════════════════════════════════╩══╩═
«                                                        0  1

This visualization shows the parametric trial state that is prepared and evaluated as part of VQE. The parameters, $\theta[n]$, are assigned a value for each iteration.

In [9]:
circ.assign_parameters(ret.optimal_parameters).draw()

┌──────────────────────┐┌───────────────────────┐     »
   0: ┤ Ry(2.64222137946031) ├┤ Rz(0.379953984461878) ├──■──»
      ├──────────────────────┤├───────────────────────┤┌─┴─┐»
   1: ┤ Ry(1.44307707650397) ├┤ Rz(-2.60468149529031) ├┤ X ├»
      └──────────────────────┘└───────────────────────┘└───┘»
c0: 2/══════════════════════════════════════════════════════»
                                                            »
«       ┌──────────────────────┐┌───────────────────────┐     »
«   0: ─┤ Ry(2.81025659773418) ├┤ Rz(0.784133105026066) ├──■──»
«      ┌┴──────────────────────┤├───────────────────────┤┌─┴─┐»
«   1: ┤ Ry(0.177366802920612) ├┤ Rz(-1.36714306392063) ├┤ X ├»
«      └───────────────────────┘└───────────────────────┘└───┘»
«c0: 2/═══════════════════════════════════════════════════════»
«                                                             »
«      ┌───────────────────────┐ ┌──────────────────────┐     »
«   0: ┤ Ry(0.106337799288022) ├─┤ Rz(1.98857084283676) ├──■──»
«      ├───────────────────────┤┌┴──────────────────────┤┌─┴─┐»
«   1: ┤ Ry(-2.03098040841544) ├┤ Rz(-1.70870039495155) ├┤ X ├»
«      └───────────────────────┘└───────────────────────┘└───┘»
«c0: 2/═══════════════════════════════════════════════════════»
«                                                             »
«       ┌─────────────────────┐ ┌───────────────────────┐┌───┐┌─┐   
«   0: ─┤ Ry(-3.291375292123) ├─┤ Rz(-6.29435226567707) ├┤ H ├┤M├───
«      ┌┴─────────────────────┴┐├───────────────────────┤├───┤└╥┘┌─┐
«   1: ┤ Ry(-1.35222181841863) ├┤ Rz(-1.42635834428133) ├┤ H ├─╫─┤M├
«      └───────────────────────┘└───────────────────────┘└───┘ ║ └╥┘
«c0: 2/════════════════════════════════════════════════════════╩══╩═
«                                                              0  1

## 2. Run on Azure Quantum backend

Now, you can connect to Azure quantum and run VQE on one of the backends.

In [67]:
from azure.quantum.qiskit import AzureQuantumProvider
provider = AzureQuantumProvider(
    resource_id = "",
    location = ""
)

In [90]:
quantinuum_emulator = provider.get_backend("quantinuum.hqs-lt-s1-sim")

### Estimate cost

You can now estimate how many credits ("HQC"/"EHQC" for Quantinuum) it will cost to run VQE. For more information about pricing, see the [Azure Quantum pricing](https://docs.microsoft.com/azure/quantum/pricing) documentation page.

In [83]:
cost = [quantinuum_backend.estimate_cost(circ, shots=1000) for circ in ready_circs]

In [84]:
for _cost in cost:
    print(_cost.estimated_total, _cost.currency_code)

16.6 EHQC
16.2 EHQC


So, given approx. 300 energy evaluations, this would give a total cost

In [87]:
energy_eval_cost = sum(_cost.estimated_total for _cost in cost)
energy_eval_cost, energy_eval_cost * 300

(32.8, 9840.0)

In [91]:
cost = [ionq_qpu_backend.estimate_cost(circ, shots=1000) for circ in ready_circs]

In [92]:
for _cost in cost:
    print(_cost.estimated_total, _cost.currency_code)

1.44 USD
1.38 USD


In [2]:
num_iterations = 300
energy_eval_cost = sum(_cost.estimated_total for _cost in cost)
energy_eval_cost, energy_eval_cost * num_iterations

(32.8, 9840.0)

To get a visual on the circuit width and depth, run:

In [78]:
print("width | depth")
for circuit in ready_circs:
    circuit = remove_idle_qwires(circuit)
    print(circuit.width(), circuit.depth())

width | depth
4 13
4 12


### Run one iteration on Quantinuum emulator via Azure Quantum

It can take a long time to run a full VQE program on hardware, because each iteration puts a circuit in the queue. To save time, run only the last iteration using the parameters we found with the Aer simulator.

In [48]:
# This is a bug that will be addressed in this PR: https://github.com/microsoft/qdk-python/pull/301
quantinuum_backend.configuration().max_shots = None

In [58]:
quantum_instance = QuantumInstance(backend=quantinuum_backend,
                                   shots=8192)
# Unset qjob config to avoid errors when running job.result()
quantum_instance._qjob_config = {}

In [59]:
optimizer = SPSA(maxiter=8192)
var_form = EfficientSU2(qubit_op.num_qubits, entanglement="linear")
vqe = VQE(qubit_op, var_form, optimizer=optimizer, initial_point=p0)

# Set the quantum instance manually
vqe.quantum_instance = quantum_instance

In [186]:
vqe._energy_evaluation(parameters=p0) + molecule.nuclear_repulsion_energy

.....................

-1.1198872295256754